# Import Required Libraries

In [ ]:
# For reading the data
import pandas as pd
import numpy as np
import sklearn
import scipy
from sklearn.preprocessing import StandardScaler
from numpy import isnan

# import the KNNimputer class
from sklearn.impute import KNNImputer

# For graphical visualization of data
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans,DBSCAN, AgglomerativeClustering
import scipy.cluster.hierarchy as sch


#from yellowbrick.cluster import SilhouetteVisualizer
from sklearn.metrics import  silhouette_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
print(pd.__version__)
print(np.__version__)
print(sklearn.__version__)
print(scipy.__version__)

# Business Problem

Perform Clustering for the World Development Measurement and identify the number of clusters formed and draw inferences

# Data Ingestion

In [ ]:
data1 = pd.read_csv(r'World_development_mesurement.csv')

In [ ]:
data1

In [ ]:
test = data1.iloc[10:,:]

In [ ]:
test

In [ ]:
data1.drop()

# Check Special Symbols in Data

In [ ]:
data1['GDP'] = data1['GDP'].str.replace('$','')
data1['GDP'] = data1['GDP'].str.replace(',','')
data1['GDP'] = data1['GDP'].astype(float)



In [ ]:

data1['Health Exp/Capita'] = data1['Health Exp/Capita'].str.replace('$','')
data1['Health Exp/Capita'] = data1['Health Exp/Capita'].str.replace(',','')
data1['Health Exp/Capita'] = data1['Health Exp/Capita'].astype(float)

In [ ]:
data1['Tourism Inbound'] = data1['Tourism Inbound'].str.replace('$','')
data1['Tourism Inbound'] = data1['Tourism Inbound'].str.replace(',','')
data1['Tourism Inbound'] = data1['Tourism Inbound'].astype(float)

In [ ]:
data1['Tourism Outbound'] = data1['Tourism Outbound'].str.replace('$','')
data1['Tourism Outbound'] = data1['Tourism Outbound'].str.replace(',','')
data1['Tourism Outbound'] = data1['Tourism Outbound'].astype(float)

In [ ]:
data1['Business Tax Rate'] = data1['Business Tax Rate'].str.replace('%','')
#data1['Business Tax Rate'] = data1['Tourism Outbound'].str.replace(',','')
data1['Business Tax Rate'] = data1['Business Tax Rate'].astype(float)


In [ ]:
# Calculating Percentage
data1['Business Tax Rate'] = data1['Business Tax Rate'] /  100

In [ ]:
data1

In [ ]:
data=data1.copy()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.info()

In [ ]:
data.dtypes

In [ ]:
data.memory_usage()

In [ ]:
round(data.describe(),2)

In [ ]:
round(data.describe().T,2)

# EDA

# Seperate Categorical and Numerical features

In [ ]:
categorical_col = [fea for fea in data.columns if data[fea].dtype == 'O']
categorical_col

In [ ]:
numerical_col = [fea for fea in data.columns if data[fea].dtype != 'O']
numerical_col

# Check the Null Values

In [ ]:
data.isna().sum()

# KNN Imputation

In [ ]:
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

In [ ]:
data.drop('Country',inplace=True,axis=1)


In [ ]:
data

In [ ]:
# fit on the dataset
imputer.fit(data)

In [ ]:
# transform the dataset
Xtrans = imputer.transform(data)

In [ ]:
Xtrans

In [ ]:

# knn imputation transform 

sum(isnan(Xtrans).flatten())

In [ ]:
data=pd.DataFrame(Xtrans)
data['Country']=data1['Country']

In [ ]:
## To check Duplicate Value is present or not in dataset.

data.duplicated()

In [ ]:
data.duplicated().sum()

In [ ]:
## It will give unique value w.r.t each column

data.nunique()

In [ ]:
# To check Correlation in the data

data.corr()

In [ ]:

#Constructing a heat map to visualize the correlation matrix
plt.figure(figsize=(20,20))
corrMatrix = data.corr()
sns.heatmap(corrMatrix, cbar=True, fmt='.1f', annot=True, cmap='Blues')


Analysis of Correlation Matrix

Correraletion matrix tells us which features are correlated. Whenever we having a strong correlation between the different features, we can remove those features. Instead of taking all features, we can take one feature. So we dont want multiple features which are very similar to each other.

Inference:-
    
1. BirthRate is highly correlated with Infant Mortality Rate and Population 0-14.

2. Co2 Emission, Energy Usage, GDP, Tourism Inbound and Tourism Outbound are highly correlated.

3. Life Expectancy Male is highly correlated with Life Expectancy Female.

# SKEW

In [ ]:
## To check the skewness distribution of the data.

data.skew()

Normal Distribution :- Birth Rate, Health Exp % GDP, Mobile Phone Usage, Population 0-14, 
    
Approximately Normal Distribution :- Population 65+, Population Urban
    
Positively Skewed Data :- Business Tax Rate, Co2 Emissions, Days To start business, Ease of Business, Energy Usage, GDP, Health Exp/Capita, Hours to do Tax, Infant Mortality Rate, Internet Usage, Lending Interest,  Population Total, Tourism Inbound, Tourism Outbound
    
Negatively Skewed Data :- Life Expectancy Female, Life Expectancy Male, Population 15-64
    


Distribution on the basis of skewness value:

Skewness = 0: Then normally distributed.

Skewness > 0: Then more weight in the left tail of the distribution.

Skewness < 0: Then more weight in the right tail of the distribution.

Inference:- The dataset contains more positively skewed data.

# KURTOSIS

In [ ]:
data.kurtosis()

kurtosis for normal distribution is equal to 3.

For a distribution having kurtosis < 3: It is called platykurtic.

For a distribution having kurtosis > 3, It is called leptokurtic and it signifies that it tries to produce more outliers rather than the normal distribution.

Inference :-
    
The columns which having more outliers are 

Co2 Emissions, 

Days to start Business, 

Energy Usage, GDP, 

Population Total, 

Tourism Inbound, 

Lending Interest.

In [ ]:
data['Country'].value_counts()

# Power Transformation For avoiding Skewness in data

In [ ]:
from sklearn.preprocessing import PowerTransformer
from scipy.stats import  skew,norm

# find numeric features in your dataset to transform
numeric_feats = data.dtypes[data.dtypes != "object"].index

# calculate skew of all numeric features
skewed_feats = data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)

# convert to dataframe for easier processing
skewness = pd.DataFrame({'Skew' :skewed_feats})

# print performance before transform
print("Pre: There are {} skewed numerical features to Power transform".format(skewness.shape[0]))
print("Pre", abs(skewness.Skew).mean())


# transform data
pt = PowerTransformer(method='yeo-johnson').fit(data.iloc[:,:-1])
#pt = PowerTransformer(method='yeo-johnson').fit(X)
X = pd.DataFrame(pt.transform(data.iloc[:,:-1]), index=data.iloc[:,:-1].index, columns=data.iloc[:,:-1].columns)

numeric_feats = X.dtypes[X.dtypes != "object"].index
skewed_feats = X[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew' :skewed_feats})

# print performance after transform
print("Post: There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))
print("Post", abs(skewness.Skew).mean())


# Graphical Analysis

In [ ]:
df_train=X

In [ ]:
df_train.columns=['Birth Rate', 'Business Tax Rate', 'CO2 Emissions', 
       'Days to Start Business', 'Ease of Business', 'Energy Usage', 'GDP',
       'Health Exp % GDP', 'Health Exp/Capita', 'Hours to do Tax',
       'Infant Mortality Rate', 'Internet Usage', 'Lending Interest',
       'Life Expectancy Female', 'Life Expectancy Male', 'Mobile Phone Usage',
       'Number of Records', 'Population 0-14', 'Population 15-64',
       'Population 65+', 'Population Total', 'Population Urban',
       'Tourism Inbound', 'Tourism Outbound']

# Univariate Analysis of Numerical Features Before Outlier Removal

In [ ]:
plt.figure(figsize=(15,20), facecolor='white')
plt.suptitle('Univariate Analysis of Numerical Features', fontweight="bold",fontsize=15, y=1)
for i in range(0,len(numerical_col)):
    plt.subplot(5,5,i+1)
    sns.kdeplot(x=df_train[numerical_col[i]], shade=True, color='r', data=df_train)
    plt.xlabel(numerical_col[i], fontsize=15)
    plt.tight_layout()
    
    
    


Most of the Columns are approximately Normal Distribution after the skewness removal.

# Check distribution and outliers together

Plots 2 graphs together

In [ ]:
for fea in numerical_col:
    plt.figure(figsize=(14,4))
    plt.subplot(121)
    sns.kdeplot(x=df_train[fea],color='r',data=df_train)
    plt.title("Distribution of {}".format(fea), fontweight='bold')
    
    plt.subplot(122)
    sns.boxplot(x=fea, data= df_train[numerical_col])
    plt.title("Distribution of {}".format(fea), fontweight='bold')
    plt.show()

Inference:-
    
The columns having more outliers are:-

1. Heaith Exp% GDP

2. Infant Mortality Rate

3. Business Tax Rate

4. Co2 Emissions, and other Leptokurtic columns




# Handling Outliers

In [ ]:
data = df_train

In [ ]:
for fea in numerical_col:
    IQR = data[fea].quantile(0.75)-data[fea].quantile(0.25)
    Lower_Limit = data[fea].quantile(0.25)-1.5*IQR
    Upper_Limit = data[fea].quantile(0.75)+1.5*IQR
    data[fea]=np.where(data[fea]>Upper_Limit,Upper_Limit,np.where(data[fea]<Lower_Limit,Lower_Limit,data[fea]))
#sns.boxplot(x=fea, data=data)

In [ ]:
for fea in numerical_col:
    plt.figure(figsize=(14,4))
    plt.subplot(121)
    sns.kdeplot(x=data[fea],color='r',data=data)
    plt.title("Distribution of {}".format(fea), fontweight='bold')
    
    plt.subplot(122)
    sns.boxplot(x=fea, data= data[numerical_col])
    plt.title("Distribution of {}".format(fea), fontweight='bold')
    plt.show()

# Univariate Analysis of Numerical Features After Outlier Removal

In [ ]:
plt.figure(figsize=(15,20), facecolor='white')
plt.suptitle('Univariate Analysis of Numerical Features', fontweight="bold",fontsize=15, y=1)
for i in range(0,len(numerical_col)):
    plt.subplot(5,5,i+1)
    sns.kdeplot(x=data[numerical_col[i]], shade=True, color='r', data=data)
    plt.xlabel(numerical_col[i], fontsize=15)
    plt.tight_layout()

# Q-Q Plot

In [ ]:
import statsmodels.api as sm
   
# Normal Q-Q plot
for i in data.columns:
    sm.qqplot(data[i], fit=True, line='45', c='#4C72B0')


# Standardise the data

In [ ]:
#Standardise the data

scale = StandardScaler()
data_norm = scale.fit_transform(data.iloc[:,:-1])


In [ ]:
data_norm

# PCA For avoiding Correlation

In [ ]:
pca = PCA(n_components = 15)
pca_data = pca.fit_transform(data_norm)

percent_var_explained = pca.explained_variance_/(np.sum(pca.explained_variance_))
cumm_var_explained = np.cumsum(percent_var_explained)

plt.plot(cumm_var_explained)
plt.grid()
plt.xlabel("n_components")
plt.ylabel("% variance explained")
plt.show()

In [ ]:
var=pca.explained_variance_ratio_
var

In [ ]:
cumm_var_explained

In [ ]:
pca.explained_variance_

In [ ]:
var=pca.explained_variance_ratio_
var

In [ ]:
var1=np.cumsum(np.round(var,4)*100)
var1

In [ ]:
var1[14]

In [ ]:
sum(pca.explained_variance_ratio_)

#Almost 98% of variance is explained with 15 new principal components

In [ ]:
data1['Country']

In [ ]:
df_train_pca = pd.DataFrame(pca_data)
df_train_pca["Country"] = data1['Country']

corr = df_train_pca.corr()

plt.figure(figsize = (22,20))

sns.heatmap(corr, annot = True, vmin=-1, vmax=1, cmap="YlGnBu", linewidths=.5)
plt.grid(b=True, color='#f68c1f', alpha=0.1)
plt.show()

No columns are correlated after PCA

In [ ]:
df_train_pca.columns=["PCA0","PCA1","PCA2","PCA3","PCA4","PCA5","PCA6","PCA7","PCA8","PCA9","PCA10","PCA11","PCA12","PCA13","PCA14","Country"]

In [ ]:
df_train_pca

# Summary Of EDA

1. Lot of missing values are there. Imputed with KNN.

2. All are numerical values except for Country column.

3. Mean is slightly more than median for most of the features. So it is right skewed. Skewness handled using PowerTransformer.

4. 10 columns are highly correlated. For removing correlation PCA used.

5. Most of the columns are LeptoKurtic.

6. All Leptokurtic columns and few Platykurtic columns have Outliers. Outliers are handled and plotted using box plot.

7. Standardised the data using Standard Scalar.

# Finding out the optimal number of clusters

In [ ]:
df_train_pca

In [ ]:
data_df=df_train_pca.copy()

In [ ]:
#Kmeans clustering is used to choose centroid that minimize inertia or within cluster sum-of-squares criteria

plt.figure(figsize=(10, 8))
wcss = []
for i in range(1, 15):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(df_train_pca.iloc[:,:-1])
    wcss.append(kmeans.inertia_)
plt.plot(range(1, 15), wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
!pip install yellowbrick

In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

In [ ]:
km = KMeans(random_state=42)
visualizer = KElbowVisualizer(km, k =(2,10))

In [ ]:
visualizer.fit((df_train_pca.iloc[:,:-1]))
visualizer.show()

Here, it is not clear what should be the Elbow point.

Let’s validate the value of K using the Silhouette plot

In [ ]:


fig, ax = plt.subplots(3, 2, figsize=(15,8))
for i in [2, 3, 4, 5, 6]:
    '''
    Create KMeans instances for different number of clusters
    '''
    km = KMeans(n_clusters=i, init='k-means++', n_init=10, max_iter=100, random_state=42)
    q, mod = divmod(i, 2)

     #Fit the KMeans model
    km.fit_predict(df_train_pca.iloc[:,:-1])
    '''
    Create SilhouetteVisualizer instance with KMeans instance
    Fit the visualizer
    '''
    visualizer = SilhouetteVisualizer(km, colors='yellowbrick', ax=ax[q-1][mod])



    # Calculate Silhoutte Score
    score = silhouette_score(df_train_pca.iloc[:,:-1],  km.labels_,  metric='euclidean')

    print('Silhouetter Score: %.3f' % score)
    visualizer.fit(df_train_pca.iloc[:,:-1]) 

Executing the above code will result in the following Silhouette plots for 2, 3, 4, 5, and 6 clusters. Silhoutte analysis is used to select an optimal value for n_clusters.

 Selecting optimal value for n_clusters is based on Silhoutte score and fluctuation in the size of silhoutte plots.

Here the silhoutte score is high for cluster 2 but the fluctuation is wider. The cluster 5 which has good silhoutte score and also the thickness of the silhoutte plot is more uniform.

Thus we can select optimal number of clusters as 5.

In [ ]:
data1=df_train_pca.iloc[:,:-1].copy()

In [ ]:
data2 = df_train_pca.iloc[:,:-1]

In [ ]:
import plotly.graph_objects as go  #for 3D plot

## K-means using k = 3
kmeans = KMeans(n_clusters=5)
kmeans.fit(data2)
y_kmeans = kmeans.predict(data2)

## 3D plot 

labels = kmeans.labels_
trace = go.Scatter3d(x=data2.iloc[:, 0].values, y=data2.iloc[:, 1].values, z=data2.iloc[:, 2].values, mode='markers',marker=dict(color = labels, size= 10, line=dict(color= 'black',width = 10)))
layout = go.Layout(margin=dict(l=0,r=0),height = 800,width = 800)
data = [trace]
fig = go.Figure(data = data, layout = layout)
fig.show()

# CLUSTERING ALGORITHMS

# K-MEANS

In [ ]:
data1

In [ ]:
kmeans = KMeans(n_clusters = 5, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(data1)

In [ ]:
y_kmeans

In [ ]:
cluster = list(y_kmeans)

In [ ]:
data1['cluster'] = cluster

In [ ]:
kmeans_mean_cluster = pd.DataFrame(round(data1.groupby('cluster').mean(),1))
kmeans_mean_cluster

In [ ]:
plt.scatter(data1.iloc[:, 0].values, data1.iloc[:, 1].values, c=y_kmeans, s=50, cmap='viridis')

# DBScan

In [ ]:
data_df2=data_df.iloc[:,:-1].copy()

In [ ]:
data_df2

In [ ]:
# we have 15 columns so we take min_samples as 15 and epsilon=n-1 ie 15-1 = 14

db=DBSCAN(eps=14,min_samples=15)
db.fit(data_df2)

In [ ]:
db.labels_

In [ ]:
#Assign clusters to the dataset

data_df2['clusterid_new']=db.labels_

In [ ]:
data_df2

In [ ]:
data_df2.groupby('clusterid_new').agg(['mean']).reset_index()

In [ ]:
data_df2[data_df2['clusterid_new']==0]

These are 1 cluster

In [ ]:
data_df2[data_df2['clusterid_new']==-1]

There is no Noise

# Heirarchial

In [ ]:
dendrogram = sch.dendrogram(sch.linkage(data_df.iloc[:,:-1], method='ward'))

In [ ]:
for i in range(3,10):
    hclusters=AgglomerativeClustering(n_clusters=i ,affinity='euclidean',linkage='ward')
    for j in range(5,12):
        y=pd.DataFrame(hclusters.fit_predict(data_df.iloc[:,:j]),columns=['H_clusters'])
        print("Cluster ValueCount : ")
        print(y['H_clusters'].value_counts())
        Hr_score = silhouette_score(data_df.iloc[:,:-1], hclusters.labels_, metric='euclidean')
        print(round(Hr_score,2),"for",i,"clusters and",j,"Principle components")

        

In [ ]:




for i in [3, 4, 5, 6, 7, 8, 9, 10]:
    '''
    Create KMeans instances for different number of clusters
    '''
    hclusters=AgglomerativeClustering(n_clusters=i ,affinity='euclidean',linkage='ward')
    

    
    hclusters.fit_predict(data_df.iloc[:,:-1])

    
  
    



    # Calculate Silhoutte Score
    score = silhouette_score(data_df.iloc[:,:-1],  hclusters.labels_,  metric='euclidean')

    print(i ,"Cluster")
    print('Silhouetter Score : %.3f' % score)


# 0.16 silhouette score for 4 clusters and 8 PCA features is good fit

In [ ]:
hclusters=AgglomerativeClustering(n_clusters=4,affinity='euclidean',linkage='ward')
hclusters

In [ ]:
dat = data_df.iloc[:,:8]

In [ ]:
y=pd.DataFrame(hclusters.fit_predict(dat),columns=['H_clusters'])
y['H_clusters'].value_counts()

In [ ]:

dat['H_cluster']=hclusters.labels_
dat

In [ ]:
dat.groupby('H_cluster').agg(['mean']).reset_index()

In [ ]:
plt.scatter(dat.iloc[labels==0, 0].values,dat.iloc[labels==0, 1].values, s=50, marker='o', color='red')
plt.scatter(dat.iloc[labels==1, 0].values, dat.iloc[labels==1, 1].values, s=50, marker='o', color='blue')
plt.scatter(dat.iloc[labels==2, 0].values, dat.iloc[labels==2, 1].values, s=50, marker='o', color='green')
plt.scatter(dat.iloc[labels==3, 0].values, dat.iloc[labels==3, 1].values, s=50, marker='o', color='yellow')

In [ ]:
import pickle
pickle_out = open("dev-model.pkl","wb")
pickle.dump(hclusters, pickle_out)
pickle_out.close()

In [ ]:
pickled_model = pickle.load(open('dev-model.pkl', 'rb'))